In [ ]:
! pip install editdistance

### Preparing file for the inference to Konstruktor


In [ ]:
import pandas as pd
import ast
from tqdm import tqdm
import editdistance
import json


train = pd.read_csv('test.tsv', sep='\t')

In [ ]:
unique_questions_test = train.question.unique()

unique_questions_dict_test = dict()
for i in range(len(unique_questions_test)):
    unique_questions_dict_test[unique_questions_test[i]] = i

for i in range(len(train)):
    train.loc[i, 'question_id'] = unique_questions_dict_test[train.loc[i, 'question']]

In [ ]:
for i in tqdm(range(len(train))):

        train_subset = train.query('sample_id == @i').reset_index(drop=True)

        cur_dict = dict()
        cur_dict_ents_names = dict()
        relations = []
        cur_dict_ents_rels = dict()
        for j in range(len(train_subset)):
            nodes = ast.literal_eval(train_subset.loc[j, 'graph'])['nodes']
            links = ast.literal_eval(train_subset.loc[j, 'graph'])['links']

            for k in range(len(nodes)):
                if nodes[k]['type'] == 'QUESTIONS_ENTITY':
                    cur_dict[nodes[k]['id']] = nodes[k]['label']
                    cur_dict_ents_names[nodes[k]['label']] = nodes[k]['name_']
            for q in range(len(links)):
                source = links[q]['source']
                relation = links[q]['label']
                if source in cur_dict:
                    if cur_dict[source] in cur_dict_ents_rels:
                        cur_dict_ents_rels[cur_dict[source]].append(relation)
                    else:
                         cur_dict_ents_rels[cur_dict[source]] = [relation]

        for k in range(len(cur_dict_ents_rels.keys())):
            cur_dict_ents_rels[list(cur_dict_ents_rels.keys())[k]]= list(set(cur_dict_ents_rels[list(cur_dict_ents_rels.keys())[k]]))

        train.loc[i, 'ques_ent_rels'] = str(cur_dict_ents_rels)


100%|██████████| 10961/10961 [00:48<00:00, 224.56it/s]


In [ ]:
for i in tqdm(range(len(train))):

        train_subset = train.query('sample_id == @i').reset_index(drop=True)

        cur_dict = dict()
        cur_dict_ents_names = dict()
        relations = []
        cur_dict_ents_rels = dict()
        for j in range(len(train_subset)):
            nodes = ast.literal_eval(train_subset.loc[j, 'graph'])['nodes']
            links = ast.literal_eval(train_subset.loc[j, 'graph'])['links']

            for k in range(len(nodes)):
                if nodes[k]['type'] == 'QUESTIONS_ENTITY':
                    cur_dict[nodes[k]['id']] = nodes[k]['label']
                    cur_dict_ents_names[nodes[k]['label']] = nodes[k]['name_']
            for q in range(len(links)):
                source = links[q]['source']
                target = links[q]['target']
                relation = links[q]['label']
                if target in cur_dict:
                    if cur_dict[target] in cur_dict_ents_rels:
                        cur_dict_ents_rels[cur_dict[target]].append(relation)
                    else:
                         cur_dict_ents_rels[cur_dict[target]] = [relation]
        for k in range(len(cur_dict_ents_rels.keys())):
            cur_dict_ents_rels[list(cur_dict_ents_rels.keys())[k]]= list(set(cur_dict_ents_rels[list(cur_dict_ents_rels.keys())[k]]))

        if train.loc[i, 'ques_ent_rels'] == "{}":
            train.loc[i, 'ques_ent_rels'] = str(cur_dict_ents_rels)

100%|██████████| 10961/10961 [00:38<00:00, 281.44it/s]


In [ ]:
train_triples = pd.DataFrame(columns = ["question", "questionEntity",  "questionEntityId",
                                        "ques_ent_rels", "question_id"])

for k in tqdm(range(1000)):
    train_subset = train.query("question_id == @k").reset_index(drop=True)

    cur_dict = dict()

    for j in range(len(train_subset)):
      keys = list(ast.literal_eval(train_subset.loc[j, "ques_ent_rels"]).keys())
      for key in keys:
        if key in cur_dict and ast.literal_eval(train_subset.loc[j, "ques_ent_rels"])[key][0] not in cur_dict[key]:
          cur_dict[key].append(ast.literal_eval(train_subset.loc[j, "ques_ent_rels"])[key][0])
        elif key not in cur_dict :
          cur_dict[key] = ast.literal_eval(train_subset.loc[j, "ques_ent_rels"])[key]
    train_triples.loc[k, "question"] = train_subset.loc[0, 'question']
    train_triples.loc[k, "questionEntity"] = train_subset.loc[0, "questionEntity"]
    train_triples.loc[k, "questionEntityId"] = train_subset.loc[0, "questionEntityId"]
    train_triples.loc[k, "ques_ent_rels"] = str(cur_dict)
    train_triples.loc[k, "question_id"] = train_subset.loc[0, "question_id"]

100%|██████████| 1000/1000 [00:05<00:00, 173.14it/s]


In [ ]:
for p in range(len(train_triples)):
  if None in ast.literal_eval(train_triples.loc[p, "ques_ent_rels"]):
      my_dict =  ast.literal_eval(train_triples.loc[p, "ques_ent_rels"])
      new_dict = my_dict.pop(None)
      train_triples.loc[p, "ques_ent_rels"] = str(my_dict)

In [ ]:
for k in tqdm(range(len(train_triples))):
   cur_dict = dict()
   ques_ents_lbl = train_triples.loc[k, 'questionEntity'].strip().split(', ')
   ques_ents_id = train_triples.loc[k, 'questionEntityId'].strip().split(', ')
   for q in range(len(ques_ents_lbl)):
      cur_dict[ques_ents_lbl[q]] = ques_ents_id[q]
   train_triples.loc[k, 'ents_dict'] = str(cur_dict)

100%|██████████| 1000/1000 [00:00<00:00, 6065.52it/s]


In [ ]:
for j in tqdm(range(len(train_triples))):

    cur_dict = ast.literal_eval(train_triples.loc[j, 'ques_ent_rels'])
    ents_dict = ast.literal_eval(train_triples.loc[j, 'ents_dict'])
    keys = list(ast.literal_eval(train_triples.loc[j, 'ques_ent_rels']).keys())
    keys_dict = list(ents_dict.keys())
    for key in keys:
      vals = cur_dict[key]
      del cur_dict[key]

      for key_dict in keys_dict:
        if key_dict.lower() == key.lower():
          cur_dict[ents_dict[key_dict]] = vals

    train_triples.loc[j, 'ques_ent_rels_id'] = str(cur_dict)

100%|██████████| 1000/1000 [00:00<00:00, 3155.34it/s]


In [ ]:
train_triples_by_one = pd.DataFrame(columns = ["question", "questionEntity",  "questionEntityId",
                                               "ques_ent_rels", "question_id"])

In [ ]:
train_triples.loc[840, 'ques_ent_rels'] = str({'United States': ['head of government', 'country']})

In [ ]:
train_triples.loc[879, 'ques_ent_rels'] = str({'United States': ['contains the administrative territorial entity', 'country'],
                                               'Texas': ['country', 'described by source']
                                               })

In [ ]:
cur_ind = 0

for i in tqdm(range(1000)):
  train_ids = list(ast.literal_eval(train_triples.loc[i, 'ques_ent_rels']).keys())

  for j in range(len(train_ids)):

    cur_ques_ent = train_ids[j]
    cur_ques_ent_id = train_triples.loc[i, 'questionEntityId'].split(', ')
    ent_rels = ast.literal_eval(train_triples.loc[i, 'ques_ent_rels'])[train_ids[j]]
    train_triples_by_one.loc[cur_ind, 'question'] = train_triples.loc[i, 'question']
    train_triples_by_one.loc[cur_ind, 'questionEntity'] = cur_ques_ent
    train_triples_by_one.loc[cur_ind, 'questionEntityId'] = cur_ques_ent_id[j]
    train_triples_by_one.loc[cur_ind, 'ques_ent_rels'] = str({cur_ques_ent_id[j]: ent_rels})
    train_triples_by_one.loc[cur_ind, 'question_id'] = train_triples.loc[i, 'question_id']
    cur_ind += 1

100%|██████████| 1000/1000 [00:01<00:00, 779.85it/s]


In [ ]:
for i in range(len(train_triples_by_one)):
   cur_ent = train_triples_by_one.loc[i, 'questionEntity']
   start_ind =  train_triples_by_one.loc[i, 'question'].find(cur_ent)
   train_triples_by_one.loc[i, 'question_rank'] = train_triples_by_one.loc[i, 'question'][:start_ind]+'<e>'+train_triples_by_one.loc[i, 'question'][start_ind+len(cur_ent):]

In [ ]:
for i in range(len(train_triples_by_one)):

    key = list(ast.literal_eval(train_triples_by_one.loc[i, 'ques_ent_rels']).keys())[0]
    train_triples_by_one.loc[i, 'relations'] = '; '.join(ast.literal_eval(train_triples_by_one.loc[i, 'ques_ent_rels'])[key])

In [ ]:
question_rank = []
path = []
scores_en_rank = []
origin_question = []
relations = []

In [ ]:
for i in range(len(train_triples_by_one)):

    question_rank.append(train_triples_by_one.loc[i, 'question_rank'])
    origin_question.append(train_triples_by_one.loc[i, 'question'])
    path.append(ast.literal_eval(train_triples_by_one.loc[i, 'ques_ent_rels']))
    relations.append(train_triples_by_one.loc[i, 'relations'].split('; '))
    scores_en_rank.append([1])

In [ ]:
test_json = dict()
test_json["questions"]= question_rank
test_json["paths"]= path
test_json["scores"]= scores_en_rank
test_json["origin_questions"]=origin_question
test_json["relations"]= relations

In [ ]:
len(test_json['questions'])

1453

In [ ]:
1467-1388

79

In [ ]:
with open('ranking_textgraphs_june.json', 'w') as f:
    json.dump(test_json, f,ensure_ascii=False)

### Extracting predicted relations from Konstrukor

In [ ]:
f = open('/content/result_ranking_june_upd.json')
res_ranking = json.load(f)

In [ ]:
def best_label(res, line):
    entity = -1000
    score_b = -1000
    label = -1000
    for key in res['re_scores'][line].keys():
        for j in range(len(res['re_scores'][line][key])):
            if score_b < res['re_scores'][line][key][j][1]:
                score_b = res['re_scores'][line][key][j][1]
                label = res['re_scores'][line][key][j][0]
                entity = key
    return [entity, label]

In [ ]:
len(res_ranking['questions']) == len(train_triples_by_one)

True

In [ ]:
for i in tqdm(range(len(train_triples_by_one))):
  subject_ranking, relation_ranking = best_label(res_ranking, i)
  train_triples_by_one.loc[i, 'subject_pred'] = subject_ranking
  train_triples_by_one.loc[i, 'relation_pred'] = relation_ranking

100%|██████████| 1453/1453 [00:00<00:00, 5856.33it/s]


In [ ]:
train['correct_path'] = 0

In [ ]:
for i in tqdm(range(len(train))):

    cur_dict = dict()
    ques_id = train.loc[i, 'question_id']
    train_subset = train_triples_by_one.query('question_id == @ques_id').reset_index(drop=True)
    for j in range(len(train_subset)):
        cur_dict[train_subset.loc[j,'questionEntity']] = train_subset.loc[j,'relation_pred']
    for key in cur_dict:
      if key in list(ast.literal_eval(train.loc[i, 'ques_ent_rels']).keys()):
        if cur_dict[key] in ast.literal_eval(train.loc[i, 'ques_ent_rels'])[key]:
          train.loc[i, 'correct_path'] += 1
      else:
          train_key = list(ast.literal_eval(train.loc[i, 'ques_ent_rels']).keys())
          for e in range(len(train_key)):
            cur_train_key = train_key[e]
            if pd.notna(key) and pd.notna(cur_train_key):
              if editdistance.eval(key.lower(), cur_train_key.lower()) <= 3:
                if cur_dict[key] in ast.literal_eval(train.loc[i, 'ques_ent_rels'])[cur_train_key]:
                  train.loc[i, 'correct_path'] += 1


100%|██████████| 10961/10961 [00:34<00:00, 315.14it/s]


In [ ]:
train.to_csv('textgraphs_konstruktor_pred.csv', index = False)